In [3]:
# import PinkRig ephys processng tools 
import glob,sys,json,shutil
from pathlib import Path 

from pykilosort import run, add_default_handler, neuropixel_probe_from_metafile

# I really wish I could shorten this line
pinkRig_path= glob.glob(r'C:\Users\*\Documents\Github\PinkRigs')
pinkRig_path = Path(pinkRig_path[0])
sys.path.insert(0, (pinkRig_path.__str__()))

from Analysis.helpers.queryExp import queryExp


# recomute should be able to take 
# err: files that errored 

sessions = queryExp(mice2check='AV014',days2check='2022-06-21')

recompute_errored_sorting = True

ephys_path = Path(sessions.iloc[0].expFolder).parent / 'ephys'
ephys_recordings = list(ephys_path.glob('**\*ap.meta'))

# for each recording resort if not already sorted 

for rec in ephys_recordings: 
    is_sorted = (rec.parent / r'pyKS\output\cluster_group.tsv').is_file()
    if not is_sorted: 
        # sort
        input_dir = rec    
        print(input_dir)  



        # check if sorting was tried already
        is_tried = (rec.parent / r'pyKS\output\.kilosort').is_dir()

        if (is_tried & recompute_errored_sorting) or not is_tried: 
            try:             
                # read the metadata and get the x- and ypos of channels                     

                channel_map  = neuropixel_probe_from_metafile(list((input_dir.parent).glob('*.meta'))[0] )

                output_dir = input_dir.parents[0] / 'pyKS'
                output_dir.mkdir(parents=False, exist_ok=True)        

                add_default_handler(level='INFO') # print output as the algorithm runs
                # find the compressed file of the same name 
                input_dir = list((input_dir.parent).glob('*.cbin'))[0] 
                run(input_dir, probe=channel_map, low_memory=True, dir_path=output_dir)
                # if there was a previous error message, remove it

                err_message_file = rec.parent / r'pyKS\pyKS_error.json'
                
                if err_message_file.is_file(): 
                    err_message_file.unlink()

            except:
                exc_type, exc_obj, exc_tb = sys.exc_info()
        
                errdict = {
                    'err_type:': str(exc_type), 
                    'err_message': str(exc_obj),
                    'traceback': str(exc_tb)
                    }

                errmessage = json.dumps(errdict)
                
                errfile = open(output_dir / 'pyKS_error.json',"w")
                errfile.write(errmessage)
                errfile.close()

                # delete temp folders before Michael kicks me out
                is_tried = (rec.parent / r'pyKS\output\.kilosort').is_dir()
                if is_tried:
                    shutil.rmtree(rec.parent / r'pyKS\output\.kilosort') 



16:50:59.136 [I] utils:279            Loaded data with 385 channels, 145360206 samples


\\zinu.cortexlab.net\Subjects\AV014\2022-06-21\ephys\AV014_2022-06-21_ActivePassive_g0\AV014_2022-06-21_ActivePassive_g0_imec0\AV014_2022-06-21_ActivePassive_g0_t0.imec0.ap.meta


16:50:59.141 [I] utils:567            Starting step whitening_matrix.


Computing the whitening matrix:   0%|          | 0/89 [00:00<?, ?it/s]

16:54:34.889 [I] preprocess:243       Computed the whitening matrix.
16:54:34.891 [I] utils:578            Step `whitening_matrix` took 215.75s.
16:54:35.033 [I] utils:567            Starting step preprocess.
16:54:35.034 [I] preprocess:402       Loading raw data and applying filters.


...

Setting cutoff:   0%|          | 0/244 [00:00<?, ?it/s]

19:54:47.582 [I] utils:578            Step `cutoff` took 14.22s.
19:54:47.582 [I] utils:578            Step `cutoff` took 14.22s.
19:54:47.582 [I] utils:578            Step `cutoff` took 14.22s.
19:54:47.582 [I] utils:578            Step `cutoff` took 14.22s.
19:54:47.644 [I] main:270             1842991 spikes after cutoff.
19:54:47.644 [I] main:270             1842991 spikes after cutoff.
19:54:47.644 [I] main:270             1842991 spikes after cutoff.
19:54:47.644 [I] main:270             1842991 spikes after cutoff.
19:54:47.647 [I] main:271             Found 93 good units.
19:54:47.647 [I] main:271             Found 93 good units.
19:54:47.647 [I] main:271             Found 93 good units.
19:54:47.647 [I] main:271             Found 93 good units.
19:54:47.649 [I] main:274             Saving results to phy.
19:54:47.649 [I] main:274             Saving results to phy.
19:54:47.649 [I] main:274             Saving results to phy.
19:54:47.649 [I] main:274             Saving results 

Computing templates:   0%|          | 0/242 [00:00<?, ?it/s]

Saving template and PC features:   0%|          | 0/41 [00:00<?, ?it/s]

19:55:09.062 [I] utils:578            Step `output` took 21.41s.
19:55:09.062 [I] utils:578            Step `output` took 21.41s.
19:55:09.062 [I] utils:578            Step `output` took 21.41s.
19:55:09.062 [I] utils:578            Step `output` took 21.41s.
19:55:11.314 [I] utils:578            Step `whitening_matrix` took 23.62s.
19:55:11.314 [I] utils:578            Step `whitening_matrix` took 23.62s.
19:55:11.314 [I] utils:578            Step `whitening_matrix` took 23.62s.
19:55:11.314 [I] utils:578            Step `whitening_matrix` took 23.62s.
19:55:11.316 [I] utils:578            Step `preprocess` took 479.47s.
19:55:11.316 [I] utils:578            Step `preprocess` took 479.47s.
19:55:11.316 [I] utils:578            Step `preprocess` took 479.47s.
19:55:11.316 [I] utils:578            Step `preprocess` took 479.47s.
19:55:11.318 [I] utils:578            Step `learn` took 885.34s.
19:55:11.318 [I] utils:578            Step `learn` took 885.34s.
19:55:11.318 [I] utils:578    